In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier # 분류나무
from sklearn.tree import DecisionTreeRegressor # 회귀나무

In [2]:
df = pd.read_csv("classification_data_01.csv")
df.head(2)

Variance  Skewness  Curtosis  Entropy  Class
0    3.6216    8.6661   -2.8073 -0.44699      0
1    4.5459    8.1674   -2.4586 -1.46210      0

In [3]:
model = DecisionTreeClassifier(max_depth = 3,
                               random_state = 123).fit(X = df.drop("Class", axis = 1),
                                                       y = df["Class"])

In [4]:
y_pred = model.predict(df.drop("Class", axis = 1))
y_pred

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [6]:
pd.crosstab(df["Class"], y_pred)

col_0    0    1
Class          
0      723   39
1       45  565

In [5]:
model.feature_importances_

array([0.75179684, 0.17772838, 0.07047477, 0.        ])

In [ ]:
# DecisionTreeRegressor()

In [9]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
iris = pd.read_csv("iris_xlsx.csv")
iris.head(2)

In [12]:
model = GaussianNB().fit(X = iris.drop("Species", axis = 1),
                         y = iris["Species"])

In [14]:
pred1 = model.predict(iris.drop("Species", axis = 1))
pred1[:3]

array(['setosa', 'setosa', 'setosa'], dtype='<U10')

In [15]:
pred2 = model.predict_proba(iris.drop("Species", axis = 1))
pred2[:3, ]

array([[1.00000000e+00, 1.35784265e-18, 7.11283512e-26],
       [1.00000000e+00, 1.51480769e-17, 2.34820051e-25],
       [1.00000000e+00, 1.07304179e-18, 2.34026774e-26]])

In [16]:
df_pred2 = pd.DataFrame(pred2,
                        columns = iris["Species"].unique())
df_pred2.head(2)

setosa    versicolor     virginica
0     1.0  1.357843e-18  7.112835e-26
1     1.0  1.514808e-17  2.348201e-25

In [19]:
df_theta = pd.DataFrame(model.theta_,
                        index = iris["Species"].unique(),
                        columns = iris.columns[:4])
df_theta

Sepal.Length  Sepal.Width  Petal.Length  Petal.Width
setosa             5.006        3.428         1.462        0.246
versicolor         5.936        2.770         4.260        1.326
virginica          6.588        2.974         5.552        2.026

In [21]:
items = ['Milk@Onion@Nutmeg@Kidney Beans@Eggs@Yogurt',
         'Dill@Onion@Nutmeg@Kidney Beans@Eggs@Yogurt',
         'Milk@Apple@Kidney Beans@Eggs',
         'Milk@Unicorn@Corn@Kidney Beans@Yogurt',
         'Corn@Onion@Onion@Kidney Beans@Ice cream@Eggs']

In [23]:
df = pd.DataFrame({"tran": range(5),
                   "items": items})
df

tran                                         items
0     0    Milk@Onion@Nutmeg@Kidney Beans@Eggs@Yogurt
1     1    Dill@Onion@Nutmeg@Kidney Beans@Eggs@Yogurt
2     2                  Milk@Apple@Kidney Beans@Eggs
3     3         Milk@Unicorn@Corn@Kidney Beans@Yogurt
4     4  Corn@Onion@Onion@Kidney Beans@Ice cream@Eggs

In [62]:
df_items = df["items"].str.split("@", expand = True).reset_index()
df_items.head()

index     0        1             2             3          4       5
0      0  Milk    Onion        Nutmeg  Kidney Beans       Eggs  Yogurt
1      1  Dill    Onion        Nutmeg  Kidney Beans       Eggs  Yogurt
2      2  Milk    Apple  Kidney Beans          Eggs       None    None
3      3  Milk  Unicorn          Corn  Kidney Beans     Yogurt    None
4      4  Corn    Onion         Onion  Kidney Beans  Ice cream    Eggs

In [61]:
df_items_melt = df_items.melt(id_vars = "index")
df_items_melt.head()

index variable value
0      0        0  Milk
1      1        0  Dill
2      2        0  Milk
3      3        0  Milk
4      4        0  Corn

In [63]:
df_items_melt.columns = ["tran", "item_cnt", "item"]
df_items_melt.head()

tran item_cnt  item
0     0        0  Milk
1     1        0  Dill
2     2        0  Milk
3     3        0  Milk
4     4        0  Corn

In [64]:
df_items_melt["item_cnt"] = df_items_melt["item_cnt"].astype("int")

In [66]:
df_items_pivot = df_items_melt.pivot_table(index = "tran", columns = "item",
                                           values = "item_cnt", fill_value = -1,
                                           aggfunc= "sum")
df_items_pivot

item  Apple  Corn  Dill  Eggs  Ice cream  Kidney Beans  Milk  Nutmeg  Onion  \
tran                                                                          
0        -1    -1    -1     4         -1             3     0       2      1   
1        -1    -1     0     4         -1             3    -1       2      1   
2         1    -1    -1     3         -1             2     0      -1     -1   
3        -1     2    -1    -1         -1             3     0      -1     -1   
4        -1     0    -1     5          4             3    -1      -1      3   

item  Unicorn  Yogurt  
tran                   
0          -1       5  
1          -1       5  
2          -1      -1  
3           1       4  
4          -1      -1

In [56]:
for col in df_items_pivot.columns:
    df_items_pivot[col] = np.where(df_items_pivot[col] >= 0, True, False)
df_items_pivot

item  Apple   Corn   Dill   Eggs  Ice cream  Kidney Beans   Milk  Nutmeg  \
tran                                                                       
0     False  False  False   True      False          True   True    True   
1     False  False   True   True      False          True  False    True   
2      True  False  False   True      False          True   True   False   
3     False   True  False  False      False          True   True   False   
4     False   True  False   True       True          True  False   False   

item  Onion  Unicorn  Yogurt  
tran                          
0      True    False    True  
1      True    False    True  
2     False    False   False  
3     False     True    True  
4      True    False   False

In [68]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [69]:
df = pd.read_csv("iris_xlsx.csv")
df.head(1)

Sepal.Length  Sepal.Width  Petal.Length  Petal.Width Species
0           5.1          3.5           1.4          0.2  setosa

In [70]:
X = df.drop("Species", axis = 1)
X_nor = StandardScaler().fit_transform(X)
df_X_nor = pd.DataFrame(X_nor, columns = X.columns)
df_X_nor.head(2)

Sepal.Length  Sepal.Width  Petal.Length  Petal.Width
0     -0.900681     1.019004     -1.340227    -1.315444
1     -1.143017    -0.131979     -1.340227    -1.315444

In [72]:
pca = PCA(n_components = 2, random_state = 123) # 생성할 주성분 개수

In [76]:
pc = pca.fit_transform(df_X_nor)
df_pc = pd.DataFrame(pc, columns = ["pc1", "pc2"])
df_pc.head(2)

pc1       pc2
0 -2.264703  0.480027
1 -2.080961 -0.674134

In [77]:
pca.explained_variance_ratio_

array([0.72962445, 0.22850762])

In [78]:
pd.Series(pca.explained_variance_ratio_).cumsum() # 누적 분산비

0    0.729624
1    0.958132
dtype: float64